In [ ]:
# https://towardsdatascience.com/hands-on-generative-adversarial-networks-gan-for-signal-processing-with-python-ff5b8d78bd28

In [1]:
import time
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [2]:
# prepare real data
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9a73ac98-d97a-427c-8ff4-73f588966b41;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 277ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/15 15:21:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 15:21:09 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
# https://stackoverflow.com/questions/68036975/valueerror-shape-must-be-at-least-rank-3-but-is-rank-2-for-node-biasadd
# config for rank error in lstm
tf.keras.backend.set_image_data_format("channels_last")

In [12]:
data = spark.read.parquet(*["s3a://sapient-bucket-trusted/prod/graph/motifs/malicious/*"]).cache()

In [14]:
start_time = time.time()
print(data.count())
print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))

2293760
 3:51PM UTC on Apr 15, 2023 --- read and cache time: 5.636562824249268 seconds ---


In [13]:
data.limit(5).toPandas()

,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,file_path_ext,timestamp_bins
0,1005022456434-2,e1,259837a5-3ec5-4c8b-9f7b-d5cfb273ea8a,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 17:15:22.372,PROCESS,CREATE,SysClient0201.systemia.com,None,None,...,other,None,None,None,None,None,None,1,None,NaN
1,1005022458026-2,e1,aa59a8ea-1863-47c1-b622-9edc216358e8,a52f6519-6265-4437-9f6c-3402fb245cc7,2019-09-23 17:15:00.330,PROCESS,CREATE,SysClient0201.systemia.com,None,None,...,other,None,None,None,None,None,None,1,None,NaN
2,1022202247098-2,e1,94b7edf1-4a91-46f5-88f9-abb47202adf7,281edc04-1ee7-4f6c-b274-cdc16b30da87,2019-09-24 15:40:44.856,FILE,READ,SysClient0501.systemia.com,None,None,...,None,None,\Device\HarddiskVolume1\Windows\WinSxS\amd64_m...,None,None,None,None,1,empty,NaN
3,1022202247388-2,e1,445b8323-6c31-4842-8eee-924a621c4c06,281edc04-1ee7-4f6c-b274-cdc16b30da87,2019-09-24 17:05:13.604,FILE,READ,SysClient0501.systemia.com,None,None,...,None,None,\Device\HarddiskVolume1\Windows\WinSxS\Manifes...,None,None,None,None,1,other,NaN
4,1022202247465-2,e1,b08fc8e4-4325-4f2b-ba2f-3a9be6c6d3f7,281edc04-1ee7-4f6c-b274-cdc16b30da87,2019-09-24 15:32:46.909,FILE,READ,SysClient0501.systemia.com,None,None,...,None,None,\Device\HarddiskVolume1\Windows\assembly\Nativ...,None,None,None,None,1,other,NaN


In [ ]:
# data.groupBy('Trace').agg( count('event')).orderBy(col('count(event)').desc()).show(5)

In [ ]:
def prepare_real_samples(ds, lc=300):
    # Calculate the number of malicious and non-malicious records
    malicious_ds = ds.filter(col("malicious") == 1).limit(lc)
    ds_events = malicious_ds.select('event_sequence').rdd.flatMap(lambda x: x).collect()
    train_sequences = tokenizer.texts_to_sequences(ds_events)
    train_padded = tf.keras.utils.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    train_ds = tf.reshape(train_padded, (-1, len(train_padded), 6))
    train_labels_ds = ones((1, input_len, 6, 1, 1))
    return train_ds, train_labels_ds

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
ds_all_events = data.select('event_sequence').rdd.flatMap(lambda x: x).collect()
tokenizer.fit_on_texts(ds_all_events)

In [ ]:
x_real, y_real = prepare_real_samples(data, lc=input_len)